In [1]:
import hepi
print(hepi.__version__)
import smpl
import numpy as np
import hepi.resummino as rs
import hepi.util as util
import matplotlib.pyplot as plt
rs.set_path("~/git/resummino")
print (rs.get_path())

0.1.6.18+dirty
~/git/resummino/


# aNNLO+NNLL wino N2C1

In [5]:

params = [
    "wino_N2C1.in",
]
pss = [
    (1000022,+1000024),
    (1000022,-1000024),
     ]
for energy  in [13000,13600]:
    hepi.set_output_dir("output_" + str(energy) + "_wino_N2C1/")
    for pa,pb in pss:
        for param in params:
            i = hepi.Input(hepi.Order.aNNLO_PLUS_NNLL,energy,pa,pb,param,"PDF4LHC21_mc","PDF4LHC21_mc",1., 1.,id="0.0.0")
            li = [i]
            li = hepi.seven_point_scan(li)
            li = hepi.pdf_scan(li)
            rs_dl = rs.run(li,skip=True,run=False,parse=False)

FileNotFoundError: [Errno 2] No such file or directory: 'output_13000_wino_N2C1/wino_N2C1.in'